In [2]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, TensorDataset

import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: ", device)

device:  cuda


In [3]:
class neo_vgg(nn.Module):
    def __init__(self):
        super(neo_vgg, self).__init__()
        # self.fc0 = nn.Linear(3, )
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=1, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=1, padding=1)
        self.conv1_s = nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=1, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=1, padding=1)
        self.conv2_s = nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=1, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=1, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=1, padding=1)
        self.conv3_s = nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=1, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=1, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=1, padding=1)
        self.conv4_s = nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=1, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=1, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=1, padding=1)
        self.conv5_s = nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.fc1 = nn.Linear(512*7*7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096,10)
        
    def forward(self, x):
        x = self.conv1_1(x)
        x = nn.ReLU()(x)
        x = self.conv1_2(x)
        x = nn.ReLU()(x)
        x = self.conv1_s(x)
        
        x = self.conv2_1(x)
        x = nn.ReLU()(x)
        x = self.conv2_2(x)
        x = nn.ReLU()(x)
        x = self.conv2_s(x)
        
        x = self.conv3_1(x)
        x = nn.ReLU()(x)
        x = self.conv3_2(x)
        x = nn.ReLU()(x)
        x = self.conv3_3(x)
        x = nn.ReLU()(x)
        x = self.conv3_s(x)
        
        x = self.conv4_1(x)
        x = nn.ReLU()(x)
        x = self.conv4_2(x)
        x = nn.ReLU()(x)
        x = self.conv4_3(x)
        x = nn.ReLU()(x)
        x = self.conv4_s(x)
        
        x = self.conv5_1(x)
        x = nn.ReLU()(x)
        x = self.conv5_2(x)
        x = nn.ReLU()(x)
        x = self.conv5_3(x)
        x = nn.ReLU()(x)
        x = self.conv5_s(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)
        
        return x
    

In [4]:
model = neo_vgg().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
valid_size = 0.15
test_size = 0.15
batch_size = 16

In [8]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):
  
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
    ])

    if test:
        dataset = datasets.CIFAR10(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


# CIFAR100 dataset 
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=batch_size)

test_loader = data_loader(data_dir='./data',
                              batch_size=batch_size,
                              test=True)

 38%|███▊      | 64225280/170498071 [02:52<04:45, 371816.38it/s] 


KeyboardInterrupt: 